In [1]:
import numpy as np
import os, sys, shutil
import scipy.ndimage as snd
import h5py
import SimpleITK as sitk
from shutil import copy
import nibabel as nib
import skimage.morphology as morph
from skimage.feature import canny
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

/home/koriavinash1/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def imshow(*args,**kwargs):
    """ Handy function to show multiple plots in on row, possibly with different cmaps and titles
    Usage: 
    imshow(img1, title="myPlot")
    imshow(img1,img2, title=['title1','title2'])
    imshow(img1,img2, cmap='hot')
    imshow(img1,img2,cmap=['gray','Blues']) """
    cmap = kwargs.get('cmap', 'gray')
    title= kwargs.get('title','')
    if len(args)==0:
        raise ValueError("No images given to imshow")
    elif len(args)==1:
        plt.title(title)
        plt.imshow(args[0], interpolation='none')
    else:
        n=len(args)
        if type(cmap)==str:
            cmap = [cmap]*n
        if type(title)==str:
            title= [title]*n
        plt.figure(figsize=(n*5,10))
        for i in range(n):
            plt.subplot(1,n,i+1)
            plt.title(title[i])
            plt.imshow(args[i], cmap[i])
    plt.show()

In [3]:
selem = morph.disk(2)
def getWeightMap(label):
    label = np.argmax(label, axis=3)[0]
    edge = np.float32(morph.binary_dilation(canny(np.float32(label)), selem))
    weight_map = np.zeros(label.shape)
    weight_map[np.where(label>0)] = 7
    weight_map = weight_map + 1
    weight_map[np.where(edge==1.0)] = 25
#     weight_map[np.where(label == 2.0)] = 15
    return np.uint8(weight_map[None,:,:])

def downSampleImage(image):
    return np.float64(snd.interpolation.zoom(image, 0.5))

def loadDicomVolume(file_path, itk_image):
    reader = sitk.ImageSeriesReader()
    reader.SetOutputPixelType(sitk.sitkFloat32)
    dicom_names = reader.GetGDCMSeriesFileNames(file_path)
    reader.SetFileNames(dicom_names)
    itk_image = reader.Execute()
    image_vol = sitk.GetArrayFromImage(self.itk_image)
    image_vol = np.transpose(image_vol,(1,2,0))
    return np.float32(image_vol)

def oneHot(targets,n_class = 6):
    axis = targets.ndim
    ret_val = (np.arange(n_class) == np.expand_dims(targets,axis = axis)).astype(int)
    return ret_val

def histogram_equalization(arr):
    nbr_bins = 256
    imhist, bins = np.histogram(arr.flatten(), nbr_bins, normed = True)
    cdf = imhist.cumsum()
    cdf = 255 * cdf / cdf[-1]
    original_shape = arr.shape
    arr = np.interp(arr.flatten(), bins[:-1], cdf)
    out_arr = arr.reshape(original_shape)
    return out_arr

def normalize(x):
    x = np.float32(x)
    min_val = np.min(x)
    max_val = np.max(x)
    ret_val = (x - min_val) / (max_val - min_val)
    return ret_val

def downSample1(slc):
    return snd.interpolation.zoom(slc,0.5)

def makeLablel(lbl, num_class = 3):
    if num_class == 2:
        lbl[lbl==2] = 1
    
    lbl = oneHot(lbl,num_class)
    return np.uint8(lbl[None,:,:])

def get_z_minmaxforbrain(lbl):
    lbl[lbl==2] = 1
    maxes = np.max(lbl,axis =(1,2))
    nonzero_maxes = np.nonzero(maxes)[0]
    mn, mx = nonzero_maxes[0] - 10, nonzero_maxes[-1] + 10
    if mn < 0:
        mn = 0
    if mx >= lbl.shape[0]:
        mx = lbl.shape[0]-1
    return mn, mx

In [4]:
root = './raw_data/pac2018_data/'
dest = './processed_data/hdf5_file/'

if not os.path.exists(dest): os.makedirs(dest)
else: 
    shutil.rmtree(dest)
    os.makedirs(dest)
    
vols = []

for f in next(os.walk(root))[2]:
    vols.append(os.path.join(root, f))
        
vols.sort()
print len(vols), vols[10].split("/")

1792 ['.', 'raw_data', 'pac2018_data', 'PAC2018_0013.nii']


In [5]:
# label paths
labels = pd.read_csv('./raw_data/PAC2018_Covariates_Upload.csv')['Label'].as_matrix()[: len(vols)]
age = pd.read_csv('./raw_data/PAC2018_Covariates_Upload.csv')['Age'].as_matrix()[: len(vols)]
gender = pd.read_csv('./raw_data/PAC2018_Covariates_Upload.csv')['Gender'].as_matrix()[: len(vols)]
tiv = pd.read_csv('./raw_data/PAC2018_Covariates_Upload.csv')['TIV'].as_matrix()[: len(vols)]
labels.shape

(1792,)

In [12]:
for vol_path, label, a, g, t in tqdm(zip(vols,labels, age, gender, tiv)):
#     print("working on : " + vol_path)
    vol_img = sitk.ReadImage(vol_path)
    vol = sitk.GetArrayFromImage(vol_img)
    
    vol = histogram_equalization(vol)
    vol = normalize(vol)
    
    vol = np.swapaxes(vol, 1, 2)
    
#     imshow(vol[:,:,72])
    
    dest_path = os.path.join(dest, vol_path.split("/")[3][:-4] +'.hdf5')
    hp = h5py.File(dest_path,'w')
    hp.create_dataset('volume', data=vol)
    hp.create_dataset('label', data=[label])
    hp.create_dataset('gender', data=[g])
    hp.create_dataset('tiv', data=[t])
    hp.create_dataset('age', data=[a])
    hp.close()

100%|██████████| 1792/1792 [14:02<00:00,  2.13it/s]


In [8]:
df = pd.read_csv('./raw_data/PAC_info_sheet.csv')
print (np.unique(df['Scanner'].as_matrix()), np.unique(df['Gender'].as_matrix()))
scanner_1_male_df = df[(df['Scanner']==1)*df['Gender']==1.]
scanner_2_male_df = df[(df['Scanner']==2)*df['Gender']==1.]
scanner_3_male_df = df[(df['Scanner']==3)*df['Gender']==1.]

scanner_1_female_df = df[(df['Scanner']==1)*df['Gender']==2.]
scanner_2_female_df = df[(df['Scanner']==2)*df['Gender']==2.]
scanner_3_female_df = df[(df['Scanner']==3)*df['Gender']==2.]

print len(scanner_1_male_df), len(scanner_2_male_df), len(scanner_3_male_df)
print len(scanner_1_female_df), len(scanner_2_female_df), len(scanner_3_female_df)

(array([1, 2, 3]), array([1., 2.]))
354 230 154
465 357 232


In [20]:
src_path = './processed_data/hdf5_file'
dest_path = './processed_data/'

def generate_train_validate_test_set(df, name):
    """
    Split the data into 70:15:15 for train-validate-test set
    arg: path: input data path
    
    generates CSV file with slice id and corrosponding bool 
    value for train, test and validation 
    """
    
    
    SPLIT_TRAIN = 0.7
    SPLIT_VALID = 0.15    
      
    train_files = df['PAC_ID'].as_matrix()
    labels = df['Label'].as_matrix()
    total_samples = len(train_files)
    print "total number of samples: {}".format(total_samples)
    
    index = np.random.randint(0, total_samples, size = total_samples)
    train_files = train_files[index]
    labels = labels[index]
    
    train_vols = train_files[0:int(SPLIT_TRAIN*total_samples)]
    valid_vols = train_files[int(SPLIT_TRAIN*total_samples): int((SPLIT_TRAIN+SPLIT_VALID)*total_samples)]
    test_vols = train_files[int((SPLIT_TRAIN+SPLIT_VALID)*total_samples):]
    
    train_labels = labels[0:int(SPLIT_TRAIN*total_samples)]
    valid_labels = labels[int(SPLIT_TRAIN*total_samples): int((SPLIT_TRAIN+SPLIT_VALID)*total_samples)]
    test_labels = labels[int((SPLIT_TRAIN+SPLIT_VALID)*total_samples):]
    
    vols_path, train, valid, test, labels = [], [], [], [], [] # to save ids and corrosponding bool values
    
    for vol, label in zip(train_vols, train_labels):
        folder_path = os.path.join(src_path, vol+'.hdf5')    
        vols_path.append('.' + folder_path)
        train.append(True)
        valid.append(False)
        test.append(False)
        labels.append(label)
    print "Training Set Done!!"
    
    for vol, label in zip(valid_vols, valid_labels):
        folder_path = os.path.join(src_path, vol+'.hdf5')            
        vols_path.append('.' + folder_path)
        train.append(False)
        valid.append(True)
        test.append(False)
        labels.append(label)
    print "Validation Set Done!!"

    for vol, label in zip(test_vols, test_labels):
        folder_path = os.path.join(src_path, vol+'hdf5')            
        vols_path.append('.' + folder_path)
        train.append(False)
        valid.append(False)
        test.append(True)
        labels.append(label)
    print "Test Set Done!!"
    
    data = pd.DataFrame()
    data['Volume_Path'] = vols_path
    data['Labels'] = labels
    data['Training'] = train
    data['Testing'] = test
    data['Validation'] = valid
    data.to_csv(os.path.join(dest_path, name + 'train_test_split.csv'), index=False)
    print "Data Splitting Done..."
    
generate_train_validate_test_set(scanner_1_female_df, 'scanner_1_female_')

total number of samples: 465
Training Set Done!!
Validation Set Done!!
Test Set Done!!
Data Splitting Done...


In [14]:
def generate_train_validate_test_set(src_path, labels, dest_path):
    """
    Split the data into 70:15:15 for train-validate-test set
    arg: path: input data path
    
    generates CSV file with slice id and corrosponding bool 
    value for train, test and validation 
    """
    
    SPLIT_TRAIN = 0.7
    SPLIT_VALID = 0.15    
      
    train_files = np.array(next(os.walk(src_path))[2])
    total_samples = len(train_files)
    print "total number of samples: {}".format(total_samples)
    
    index = np.random.randint(0, total_samples, size = total_samples)
    train_files = train_files[index]
    labels = labels[index]
    
    train_vols = train_files[0:int(SPLIT_TRAIN*total_samples)]
    valid_vols = train_files[int(SPLIT_TRAIN*total_samples): int((SPLIT_TRAIN+SPLIT_VALID)*total_samples)]
    test_vols = train_files[int((SPLIT_TRAIN+SPLIT_VALID)*total_samples):]
    
    train_labels = labels[0:int(SPLIT_TRAIN*total_samples)]
    valid_labels = labels[int(SPLIT_TRAIN*total_samples): int((SPLIT_TRAIN+SPLIT_VALID)*total_samples)]
    test_labels = labels[int((SPLIT_TRAIN+SPLIT_VALID)*total_samples):]
    
    vols_path, train, valid, test, labels = [], [], [], [], [] # to save ids and corrosponding bool values
    
    for vol, label in zip(train_vols, train_labels):
        folder_path = os.path.join(src_path, vol)    
        vols_path.append('.' + folder_path)
        train.append(True)
        valid.append(False)
        test.append(False)
        labels.append(label)
    print "Training Set Done!!"
    
    for vol, label in zip(valid_vols, valid_labels):
        folder_path = os.path.join(src_path, vol)            
        vols_path.append('.' + folder_path)
        train.append(False)
        valid.append(True)
        test.append(False)
        labels.append(label)
    print "Validation Set Done!!"

    for vol, label in zip(test_vols, test_labels):
        folder_path = os.path.join(src_path, vol)            
        vols_path.append('.' + folder_path)
        train.append(False)
        valid.append(False)
        test.append(True)
        labels.append(label)
    print "Test Set Done!!"
    
    data = pd.DataFrame()
    data['Volume Path'] = vols_path
    data['Labels'] = labels
    data['Training'] = train
    data['Testing'] = test
    data['Validation'] = valid
    data.to_csv(os.path.join(dest_path, 'train_test_split.csv'), index=False)
    print "Data Splitting Done..."

In [8]:
generate_train_validate_test_set('./processed_data/hdf5_file', labels, './processed_data/')

total number of samples: 1792
Training Set Done!!
Validation Set Done!!
Test Set Done!!
Data Splitting Done...


In [9]:
data = pd.read_csv('./processed_data/train_test_split.csv')
data

,Volume Path,Labels,Training,Testing,Validation
0,../processed_data/hdf5_file/PAC2018_0747.hdf5,1.0,True,False,False
1,../processed_data/hdf5_file/PAC2018_1948.hdf5,2.0,True,False,False
2,../processed_data/hdf5_file/PAC2018_1905.hdf5,1.0,True,False,False
3,../processed_data/hdf5_file/PAC2018_0523.hdf5,1.0,True,False,False
4,../processed_data/hdf5_file/PAC2018_1306.hdf5,1.0,True,False,False
5,../processed_data/hdf5_file/PAC2018_1674.hdf5,1.0,True,False,False
6,../processed_data/hdf5_file/PAC2018_1386.hdf5,2.0,True,False,False
7,../processed_data/hdf5_file/PAC2018_2195.hdf5,1.0,True,False,False
8,../processed_data/hdf5_file/PAC2018_0276.hdf5,1.0,True,False,False
9,../processed_data/hdf5_file/PAC2018_0887.hdf5,1.0,True,False,False
